In [1]:
import pandas as pd
from google.cloud import bigquery
import os
import re
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/Users/sanjamekas/Documents/ekas.json'

In [2]:
bigquery_client = bigquery.Client(project='switch-vc')

### getting the dataset

In [190]:
query ="""
SELECT  
   A.uuid, A.company_name, A.category_list, A.category_group_list
FROM
  `switch-vc.founder_crunchbase.organizations` AS A
INNER JOIN
  `switch-vc.analysis.cb_orgs_final_selected_v2` AS B
ON
  A.uuid = B.cb_uuid
"""

query_job = bigquery_client.query(query)
df = query_job.to_dataframe()
print(len(df))
df.head()

37577


,uuid,company_name,category_list,category_group_list
0,42d1a741-ef81-9caf-dfee-281c9451b06e,Optaros,e-commerce|information technology|open source|...,apps|commerce and shopping|information technol...
1,4cc6fe8d-3a2b-c327-e008-69024ecb95a2,Rapid Micro Biosystems,biotechnology|manufacturing|pharmaceutical,biotechnology|health care|manufacturing|scienc...
2,42719ae6-8a13-c91d-c812-2eeb1bf6d356,SweetLabs,android|software|web apps,apps|mobile|platforms|software
3,36aab34c-1cb1-e96d-2e5a-e298d147909c,Caviar,food and beverage|food delivery|hospitality,food and beverage|transportation|travel and to...
4,074f4754-6ee9-b81f-decd-14675f244ef5,Klout,analytics|internet|social media,data and analytics|internet services|media and...


In [5]:
df.category_list[1]

'biotechnology|manufacturing|pharmaceutical'

### Traditional OLD WAY OF DOING THINGS

In [212]:
org = ['health','pharma','thera','bio','medic','life science']
lis = []
for i in range(len(df)):
    try:
        for j in org:
            if j in df.category_list[i]:
                if i not in lis:
                    lis.append(i)
    except:
        pass
print(len(lis))

6981


In [213]:
df1 = df.loc[lis]

In [214]:
df1 = df1.reset_index()
df1 = df1.drop(['index'],axis=1)
df1.head()

,uuid,company_name,category_list,category_group_list
0,4cc6fe8d-3a2b-c327-e008-69024ecb95a2,Rapid Micro Biosystems,biotechnology|manufacturing|pharmaceutical,biotechnology|health care|manufacturing|scienc...
1,a0420895-9b5d-4098-25f0-376dadd054a7,ModernCare,health care|medical|travel,health care|travel and tourism
2,66d1bf52-3753-4e40-74b0-2d0b86be12e3,Foldax,developer platform|manufacturing|medical device,health care|manufacturing|software
3,ddaacf7b-1fa1-ea57-75d2-c48bda98de28,Loop Genomics,biotechnology|genetics|information technology|...,biotechnology|health care|information technolo...
4,a2bc50f6-9fb1-5d37-5a31-84d0b9d697e9,Twentyeight-Seven,biotechnology|developer platform|therapeutics,biotechnology|health care|science and engineer...


# CORRECT REGEX

In [223]:
org = ['health','pharma','thera','bio','medic','life science']
d = []
for j in range(len(df)):
    a = df.category_list[j]
    li = []
    try:
        for i in org:
            pattern = re.compile(r"^.*"+i+"+")
            match = pattern.findall(a)
            if len(match)!=0:
                li.append(match)

        list1 = max(li)
        str1 = ''.join(list1)
        final = str1.split('|')
        d.append(j)
    
    except:
        pass

In [224]:
len(d)

6981

In [194]:
df2 = df.loc[d]

In [195]:
df2.head()

,uuid,company_name,category_list,category_group_list
1,4cc6fe8d-3a2b-c327-e008-69024ecb95a2,Rapid Micro Biosystems,biotechnology|manufacturing|pharmaceutical,biotechnology|health care|manufacturing|scienc...
21,a0420895-9b5d-4098-25f0-376dadd054a7,ModernCare,health care|medical|travel,health care|travel and tourism
22,66d1bf52-3753-4e40-74b0-2d0b86be12e3,Foldax,developer platform|manufacturing|medical device,health care|manufacturing|software
46,ddaacf7b-1fa1-ea57-75d2-c48bda98de28,Loop Genomics,biotechnology|genetics|information technology|...,biotechnology|health care|information technolo...
57,a2bc50f6-9fb1-5d37-5a31-84d0b9d697e9,Twentyeight-Seven,biotechnology|developer platform|therapeutics,biotechnology|health care|science and engineer...


In [196]:
df2 = df2.reset_index()
df2 = df2.drop(['index'],axis=1)
df2.head()

,uuid,company_name,category_list,category_group_list
0,4cc6fe8d-3a2b-c327-e008-69024ecb95a2,Rapid Micro Biosystems,biotechnology|manufacturing|pharmaceutical,biotechnology|health care|manufacturing|scienc...
1,a0420895-9b5d-4098-25f0-376dadd054a7,ModernCare,health care|medical|travel,health care|travel and tourism
2,66d1bf52-3753-4e40-74b0-2d0b86be12e3,Foldax,developer platform|manufacturing|medical device,health care|manufacturing|software
3,ddaacf7b-1fa1-ea57-75d2-c48bda98de28,Loop Genomics,biotechnology|genetics|information technology|...,biotechnology|health care|information technolo...
4,a2bc50f6-9fb1-5d37-5a31-84d0b9d697e9,Twentyeight-Seven,biotechnology|developer platform|therapeutics,biotechnology|health care|science and engineer...


---

### PART 2 BREAK

In [225]:
s = ['biotechnology','advertising','financial','thera','clothing','pharma','consult','admin','lifestyle','sustainability','travel','food','professional','transport','media','real','payment','publishing','media','sales','commerce','government','gaming','messaging','education','sport','design','agriculture']
d = []
for j in range(len(df2)):
    a = df2.category_group_list[j]
    li = []
    try:
        for i in s:
            pattern = re.compile(r"^.*"+i+"+")
            match = pattern.findall(a)
            if len(match)!=0:
                li.append(match)

        list1 = max(li)
        str1 = ''.join(list1)
        final = str1.split('|')
        d.append(j)
    
    except:
        pass

In [226]:
len(d)

4083

In [227]:
df3 = df2.loc[d]

In [228]:
df3 = df3.reset_index()
df3 = df3.drop(['index'],axis=1)
df3.head()

,uuid,company_name,category_list,category_group_list
0,4cc6fe8d-3a2b-c327-e008-69024ecb95a2,Rapid Micro Biosystems,biotechnology|manufacturing|pharmaceutical,biotechnology|health care|manufacturing|scienc...
1,a0420895-9b5d-4098-25f0-376dadd054a7,ModernCare,health care|medical|travel,health care|travel and tourism
2,ddaacf7b-1fa1-ea57-75d2-c48bda98de28,Loop Genomics,biotechnology|genetics|information technology|...,biotechnology|health care|information technolo...
3,a2bc50f6-9fb1-5d37-5a31-84d0b9d697e9,Twentyeight-Seven,biotechnology|developer platform|therapeutics,biotechnology|health care|science and engineer...
4,235cdf32-5fcd-7afa-7334-8f5a7ae7eeb0,EdeniQ,agriculture|biotechnology|fuel,agriculture and farming|biotechnology|energy|s...


---

### Part 3 break

In [229]:
s = ['hsport','tutoring','cosmetic','sms','diabetes','nutrition','dental','medicine','charity','family','wellness','hospital','consumption','women','agency','non profit','enterprise','employment','billing','clinical','trials','navigation','dietary','hospitality','elder','environ','beauty','advert','agriculture','energy','real estate','environ','news','developer','child care','consulting','rehab','nurs','government','food']

d = []
for j in range(len(df2)):
    a = df2.category_list[j]
    li = []
    try:
        for i in s:
            pattern = re.compile(r"^.*"+i+"+")
            match = pattern.findall(a)
            if len(match)!=0:
                li.append(match)

        list1 = max(li)
        str1 = ''.join(list1)
        final = str1.split('|')
        d.append(j)
    
    except:
        pass

In [230]:
len(d)

1351

In [231]:
df4 = df2.loc[d]

In [232]:
df4 = df4.reset_index()
df4 = df4.drop(['index'],axis=1)
df4.head()

,uuid,company_name,category_list,category_group_list
0,66d1bf52-3753-4e40-74b0-2d0b86be12e3,Foldax,developer platform|manufacturing|medical device,health care|manufacturing|software
1,a2bc50f6-9fb1-5d37-5a31-84d0b9d697e9,Twentyeight-Seven,biotechnology|developer platform|therapeutics,biotechnology|health care|science and engineer...
2,eb0fbe3d-caee-48fc-e0c5-0327e3f3e477,KeepLiving,health care|information services|wellness,health care|information technology
3,9f51a0b6-8cec-6f82-b854-b8df24e0eedf,Inofile,developer platform|health care|information ser...,health care|information technology|software
4,ab060e9f-e063-ea35-9ca0-f548a35d8fe0,CRESCEL,diabetes|manufacturing|pharmaceutical,health care|manufacturing


---

### combining both dfs without duplicates

In [233]:
a = pd.concat([df3,df4]).drop_duplicates().reset_index(drop=True)

In [234]:
len(a)

4702

In [235]:
a.head()

,uuid,company_name,category_list,category_group_list
0,4cc6fe8d-3a2b-c327-e008-69024ecb95a2,Rapid Micro Biosystems,biotechnology|manufacturing|pharmaceutical,biotechnology|health care|manufacturing|scienc...
1,a0420895-9b5d-4098-25f0-376dadd054a7,ModernCare,health care|medical|travel,health care|travel and tourism
2,ddaacf7b-1fa1-ea57-75d2-c48bda98de28,Loop Genomics,biotechnology|genetics|information technology|...,biotechnology|health care|information technolo...
3,a2bc50f6-9fb1-5d37-5a31-84d0b9d697e9,Twentyeight-Seven,biotechnology|developer platform|therapeutics,biotechnology|health care|science and engineer...
4,235cdf32-5fcd-7afa-7334-8f5a7ae7eeb0,EdeniQ,agriculture|biotechnology|fuel,agriculture and farming|biotechnology|energy|s...


### verifying

In [236]:
a[a.uuid=='1108ba19-5761-a120-fb85-fb8a0d0ccaac'].index.values 

array([906])

### FULLY CORRECT 
- matched output
- matched data columns
- everything is same

---